# test

## include scripts

### include notebook core

In [ ]:
. ../../../../scripts/nbs_header.ps1

### Include core functions script

In [ ]:
. ../../../../scripts/core.ps1

### Include spiral library

In [ ]:
. ../../../../deps/spiral/lib/spiral/lib.ps1

## execute project commands

### run notebook with retries using spiral supervisor

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --execute-command "../../../../deps/spiral/workspace/target/release/spiral$(_exe) dib --path test.dib --retries 3" } | Invoke-Block

00:00:00 v #1 networking.test_port_open / { port = 13806; ex = System.AggregateException: One or more errors occurred. (Connection refused) }
00:00:00 d #1 runtime.execute_with_options_async / { file_name = ../../../../deps/spiral/workspace/target/release/spiral; arguments = US5_0 "dib --path test.dib --retries 3"; options = { command = ../../../../deps/spiral/workspace/target/release/spiral dib --path test.dib --retries 3; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = None; stdin = None; trace = true; working_directory = None } }
00:00:00 v #2 > 00:00:00 d #1 spiral.main / { args = Array(MutCell(["dib", "--path", "test.dib", "--retries", "3"])) }
00:00:00 v #3 > 00:00:00 d #2 runtime.execute_with_options / { file_name = dotnet; arguments = ["repl", "--exit-after-run", "--run", "/home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.dib", "--output-path", "/home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.dib.

### parse the .dib file into .spi format with dibparser

In [ ]:
{ . ../../../../apps/parser/dist/DibParser$(_exe) test.dib spi } | Invoke-Block

00:00:00 d #1 writeDibCode / output: Spi / path: test.dib
00:00:00 d #2 parseDibCode / output: Spi / file: test.dib


### build .fsx file from .spi using supervisor

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --build-file test.spi test.fsx } | Invoke-Block

00:00:00 v #1 networking.test_port_open / { port = 13806; ex = System.AggregateException: One or more errors occurred. (Connection refused) }
00:00:00 v #2 networking.test_port_open / { port = 13806; ex = System.AggregateException: One or more errors occurred. (Connection refused) }
00:00:00 d #1 Supervisor.buildFile / takeWhileInclusive / path: test.spi / errors: [] / typeErrorCount: 0 / retry: 0 / outputContent:

00:00:00 d #2 Supervisor.buildFile / AsyncSeq.scan / path: test.spi / errors: [] / outputContentResult:  / typeErrorCount: 0 / retry: 0 / error:  / outputContent:

00:00:00 d #3 Supervisor.buildFile / takeWhileInclusive / path: test.spi / errors: [] / typeErrorCount: 0 / retry: 0 / outputContent:

00:00:00 v #4 Supervisor.sendJson / port: 13805 / json: {"FileOpen":{"spiText":"/// # test (Polyglot)\n\n/// ## main\ninl main (_args : array_base string) =...t\u003E\u0022\n","uri":"file:///home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.spi"}} / result:
00:00:00 v #

## compile and format the project

### compile project with fable targeting optimized rust

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 5.0.0-alpha.2: F# to Rust compiler (status: alpha)

Thanks to the contributor! @fdcastel
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
Project and references (1 source files) parsed in 2367ms

Started Fable compilation...

Fable compilation finished in 1086ms

./test.fsx(11,0): (11,2) warning FABLE: For Rust, support for F# static and module do bindings is disabled by default. It can be enabled with the 'static_do_bindings' feature. Use at your own risk!


### fix formatting issues in the .rs file using regex and set-content

In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
    | FixRust `
| Set-Content test.rs

### format the rust code using cargo fmt

In [ ]:
cargo fmt --

## build and test the project

### build the project in release mode using nightly rust compiler

In [ ]:
cargo build --release

   Compiling byteorder v1.5.0
   Compiling linux-raw-sys v0.4.14
   Compiling syn v2.0.90
   Compiling getrandom v0.2.15
   Compiling rand_core v0.6.4
   Compiling num-traits v0.2.19
   Compiling once_cell v1.20.2
   Compiling libm v0.2.11
   Compiling rustix v0.38.42
   Compiling wait-timeout v0.2.0
   Compiling quick-error v1.2.3
   Compiling bit-vec v0.6.3
   Compiling bit-set v0.5.3
   Compiling rand_xorshift v0.3.0
   Compiling memchr v2.7.4
   Compiling unarray v0.1.4
   Compiling nom v7.1.3
   Compiling fable_library_rust v0.1.0 (/home/runner/work/polyglot/polyglot/lib/rust/fable/fable_modules/fable-library-rust)
   Compiling zerocopy-derive v0.7.35
   Compiling tempfile v3.14.0
   Compiling rusty-fork v0.3.0
   Compiling zerocopy v0.7.35
   Compiling thiserror-impl v1.0.69
   Compiling ppv-lite86 v0.2.20
   Compiling rand_chacha v0.3.1
   Compiling rand v0.8.5
   Compiling thiserror v1.0.69
   Compiling proptest v1.5.0
   Compiling spiral_temp_test v0.0.1 (/home/runner/work/pol

### run release tests with output enabled

In [ ]:
{ cargo test --release -- --show-output } | Invoke-Block

   Compiling bitflags v2.6.0
   Compiling linux-raw-sys v0.4.14
   Compiling zerocopy v0.7.35
   Compiling fastrand v2.3.0
   Compiling once_cell v1.20.2
   Compiling wait-timeout v0.2.0
   Compiling bit-vec v0.6.3
   Compiling rustix v0.38.42
   Compiling quick-error v1.2.3
   Compiling fnv v1.0.7
   Compiling bit-set v0.5.3
   Compiling num-traits v0.2.19
   Compiling ppv-lite86 v0.2.20
   Compiling rand_xorshift v0.3.0
   Compiling minimal-lexical v0.2.1
   Compiling rand_chacha v0.3.1
   Compiling unarray v0.1.4
   Compiling lazy_static v1.5.0
   Compiling rand v0.8.5
   Compiling memchr v2.7.4
   Compiling fable_library_rust v0.1.0 (/home/runner/work/polyglot/polyglot/lib/rust/fable/fable_modules/fable-library-rust)
   Compiling nom v7.1.3
   Compiling tempfile v3.14.0
   Compiling rusty-fork v0.3.0
   Compiling proptest v1.5.0
   Compiling thiserror v1.0.69
   Compiling spiral_temp_test v0.0.1 (/home/runner/work/polyglot/polyglot/apps/spiral/temp/test)
    Finished `release` prof

### execute the binary in release mode

In [ ]:
{ . $ScriptDir/../../../../workspace/target/release/spiral_temp_test$(_exe) } | Invoke-Block

app=test
